In [1]:
import numpy as np
import pylab as plt
import quadprog as QP

In [128]:
def hat_map(x):   
    return np.array([[0,-x[2],x[1]],[x[2],0,-x[0]],[-x[1],x[0],0]])

In [3]:
def position_dynamics(tau,x,xdot,v):
    return(xdot)

In [4]:
def velocity_dynamics(tau,x,xdot,v):    
    return(-np.inner(xdot,xdot)*x-np.dot(hat_map(q)**2,v))

In [48]:
def reference(tau):
    ref_theta = 3*np.pi/4
    ref_phi = 0.25*tau
    return(np.array([np.sin(ref_theta)*np.cos(ref_phi),np.sin(ref_theta)*np.sin(ref_phi),np.cos(ref_theta)]))

In [49]:
def reference_velocity(tau):
    ref_theta = 3*np.pi/4
    ref_phi = 0.25*tau
    return(np.array([-0.25*np.sin(ref_theta)*np.sin(ref_phi),0.25*np.sin(ref_theta)*np.cos(ref_phi),np.cos(ref_theta)]))

In [50]:
def reference_acceleration(tau):
    ref_theta = 3*np.pi/4
    ref_phi = 0.25*tau
    return(np.array([-0.25**2*np.sin(ref_theta)*np.cos(ref_phi),-0.25**2*np.sin(ref_theta)*np.sin(ref_phi),np.cos(ref_theta)]))

In [51]:
def safety_center(tau,invariant):
    if invariant == 1:
        return(np.array([0,0,-1]))
    else:
        ref_theta = 4*np.pi/5
        ref_phi = 0.25*tau
        return(np.array([np.sin(ref_theta)*np.cos(ref_phi),np.sin(ref_theta)*np.sin(ref_phi),np.cos(ref_theta)]))

In [52]:
def safety_center_velocity(tau,invariant):  
    if invariant == 1:
        return(np.array([0,0,0]))
    else:
        ref_theta = 4*np.pi/5
        ref_phi = 0.25*tau
        return(np.array([-0.25*np.sin(ref_theta)*np.sin(ref_phi),0.25*np.sin(ref_theta)*np.cos(ref_phi),np.cos(ref_theta)]))

In [53]:
def safety_center_acceleration(tau,invariant):  
    if invariant == 1:
        return(np.array([0,0,0]))
    else:
        ref_theta = 4*np.pi/5
        ref_phi = 0.25*tau
        return(np.array([-0.25**2*np.sin(ref_theta)*np.cos(ref_phi),-0.25**2*np.sin(ref_theta)*np.sin(ref_phi),np.cos(ref_theta)]))

In [11]:
def position_error(tau,x):
    return(np.dot(hat_map(x)**2,reference(tau)))

In [12]:
def velocity_error(tau,x,xdot):    
    return(xdot-np.cross(np.cross(reference(tau),reference_velocity(tau)),x))

In [13]:
def safety_constraints(tau,x,invariant,i):
    if i == 0:
        g = np.cos(np.pi/12)+np.inner(x,safety_center(tau,invariant))
    else:
        if i == 1:
            g = -np.cos(np.pi/4)-np.inner(x,safety_center(tau,invariant))
    return(g)

In [14]:
def CLF(tau,x,xdot):
    V1 = 0.5*np.inner(velocity_error(tau,x,xdot),velocity_error(tau,x,xdot))
    V2 = 0.5*alpha*(1-np.inner(x,reference(tau)))
    V3 = epsilon*np.inner(position_error(tau,x),velocity_error(tau,x,xdot))
    return(V1+V2+V3)

In [15]:
def h_function(tau,x,xdot,i,invariant):
    return(gamma[i]*safety_constraints(tau,x,invariant,i) + 
           (-1)**i*(np.inner(safety_center(tau,invariant),xdot)+np.inner(safety_center_velocity(tau,invariant),x)))

In [16]:
def CBF(tau,x,xdot,i,invariant):     
    return(1/h_function(tau,x,xdot,i,invariant))

In [95]:
def feedforward_control_law(tau,x,xdot):
    u1 = np.cross(np.cross(reference(tau),reference_acceleration(tau)),x)    
    u2 = -np.dot(hat_map(x)**2,np.cross((np.cross(reference(tau),reference_velocity(tau))),xdot).T).T    
    return(u1+u2)

In [18]:
def phi_0(tau,i,invariant):    
    return((-1)**i*safety_center(tau,invariant))

In [57]:
def phi_1(tau,x,xdot,i,invariant):    
    p1 = gamma[i]*(h_function(tau,x,xdot,i,invariant)-gamma[i]*safety_constraints(tau,x,invariant,i))
    p2 = (-1)**i*(2*np.inner(xdot,safety_center_velocity(tau,invariant))+
                    np.inner(x,safety_center_acceleration(tau,invariant))-
                    np.inner(xdot,xdot)*np.inner(safety_center(tau,invariant),x))
    return(p1+p2)

In [20]:
def psi_0(tau,x,xdot):
    return(velocity_error(tau,x,xdot) + epsilon*position_error(tau,x))

In [81]:
def psi_1(tau,x,xdot):
    p11 = np.cross(np.cross(reference(tau),reference_acceleration(tau)),x)
    p12 = np.cross((np.cross(reference(tau),reference_velocity(tau))),xdot)
    P1 = -np.inner(psi_0(tau,x,xdot),p11+p12)
    P2 = -0.5*alpha*(np.inner(xdot,reference(tau))+np.inner(x,reference_velocity(tau)))
    p31 = np.cross(xdot,(np.cross(x,reference(tau))))
    p32 = np.cross(x,(np.cross(xdot,reference(tau))+np.cross(x,reference_velocity(tau))))
    P3 = np.inner(epsilon*position_error(tau,x),p31+p32)    
    return(P1+P2+P3)

In [22]:
def second_order_rk4_step(tau,x0,xdot0,v):    
    h = tau[1]-tau[0]
    k11 = h*position_dynamics(tau[0],x0,xdot0,v)
    k12 = h*velocity_dynamics(tau[0],x0,xdot0,v)
    k21 = h*position_dynamics(tau[0]+h/2,x0+k11/2,xdot0+k12/2,v)
    k22 = h*velocity_dynamics(tau[0]+h/2,x0+k11/2,xdot0+k12/2,v)
    k31 = h*position_dynamics(tau[0]+h/2,x0+k21/2,xdot0+k22/2,v)
    k32 = h*velocity_dynamics(tau[0]+h/2,x0+k21/2,xdot0+k22/2,v)
    k41 = h*position_dynamics(tau[0]+h,x0+k31,xdot0+k32,v)
    k42 = h*velocity_dynamics(tau[0]+h,x0+k31,xdot0+k32,v)
    x = x0 + (k11+2*k21+2*k31+k41)/6
    xdot = xdot0 + (k12+2*k22+2*k32+k42)/6
    return(x,xdot)

In [152]:
def geometric_control_law(tau,x,xdot,invariant):
    G = np.concatenate((np.concatenate((H,np.zeros([3,1])),axis=1),np.array([np.array([0,0,0,Lambda])])),axis=0)
    a = np.zeros([4,])     
    c1 = np.concatenate((np.array([-psi_0(tau,x,xdot)]).T,np.zeros([1,1])),axis=0)    
    c2 = np.concatenate((np.array([phi_0(tau,0,invariant)]).T,np.zeros([1,1])),axis=0)
    c3 = np.concatenate((np.array([phi_0(tau,1,invariant)]).T,np.zeros([1,1])),axis=0)
    C = np.concatenate((c1,c2,c3),axis=1)
    u_ff = feedforward_control_law(tau,x,xdot)
    b1 = np.inner(psi_0(tau,x,xdot),u_ff)+psi_1(tau,x,xdot)+eta*CLF(tau,x,xdot)    
    b2 = -(np.inner(phi_0(tau,0,invariant),u_ff)+phi_1(tau,x,xdot,0,invariant)+mu[0]*h_function(tau,x,xdot,0,invariant)**3)
    b3 = -(np.inner(phi_0(tau,1,invariant),u_ff)+phi_1(tau,x,xdot,1,invariant)+mu[1]*h_function(tau,x,xdot,1,invariant)**3)
    b = np.array([np.asscalar(b1),np.asscalar(b2),np.asscalar(b3)])    
    x,f,xu,iterations,lagrangian,iact = QP.solve_qp(G,a,C,b)
    return(x)

In [153]:
geometric_control_law(0,q0,qdot0,0)

[-8.91153551  0.59487076  8.61115074  0.        ]


In [134]:
psi_0(0,q0,qdot0).shape

(3,)

In [158]:
t0 = 0
tmax = 10
n_points = 100
alpha = 1
epsilon = 1
gamma = [1,1]

H = np.array([[2, 1, 1],[1, 2, 1],[1,1,2]])
Lambda = 10
eta = 1
mu = [1,1]
invariant = 1

In [124]:
theta0 = 11*np.pi/15 
phi0 = 0
q0 = np.array([np.sin(theta0)*np.cos(phi0),np.sin(theta0)*np.sin(phi0),np.cos(theta0)])
qdot0 = np.array([0,0,0])

In [156]:
t = np.linspace(t0,tmax,n_points)
q = np.zeros([t.shape[0],3])
qdot = np.zeros([t.shape[0],3])
qd = np.zeros([t.shape[0],3])
qd_dot = np.zeros([t.shape[0],3])
q[0,:] = q0
qdot[0,:] = qdot0
qd[0,:] = reference(t[0])
qd_dot[0,:] = reference_velocity(t[0])

In [39]:
for i in range(1,t.size):
    u_ff = feedforward_control_law(tau,x,xdot)
    u_fb = geometric_control_law(tau,x,xdot,invariant)
    u = u_ff+u_fb
    q[i,:],qdot[i,:] = second_order_rk4_step(t[i:i-1],q[i-1,:],qdot[i-1,:],u)
    qd[i,:] = reference(t[i])
    qd_dot[i,:] = reference_velocity(t[i])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [37]:
t.size

100